# Prepare forest management data (Lesiv)

In [ ]:
# Libraries
import os, time, shutil, rioxarray
import numpy as np
import xarray as xr
from scipy import ndimage
from dask.distributed import Lock
import matplotlib.pyplot as plt

In [ ]:
# Directories
dir_data =  '../data/'
dir01 = '../paper_deficit/output/01_prep/'

---

In [ ]:
# Libraries
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import dask

# Initialize dask
cluster = SLURMCluster(
    queue='compute',                      # SLURM queue to use
    cores=24,                             # Number of CPU cores per job
    memory='256 GB',                      # Memory per job
    account='bm0891',                     # Account allocation
    interface="ib0",                      # Network interface for communication
    walltime='00:30:00',                  # Maximum runtime per job
    local_directory='../dask/',           # Directory for local storage
    job_extra_directives=[                # Additional SLURM directives for logging
        '-o ../dask/LOG_worker_%j.o',     # Output log
        '-e ../dask/LOG_worker_%j.e'      # Error log
    ]
)

# Scale dask cluster
cluster.scale(jobs=2)

# Configurate dashboard url
dask.config.config.get('distributed').get('dashboard').update(
    {'link': '{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'}
)

# Create client
client = Client(cluster)

client

In [ ]:
# prepare lesiv (forest management) data
def prepare_lesiv_nat():
    """ 
    Prepare Lesiv 2022 natural forest data
    """
    #read forest management data
    file_path = dir_data + 'lesiv2022/FML_v3-2_with-colorbar.tif'
    
    da_lesiv = rioxarray.open_rasterio(file_path, 
                                       chunks=dict(y=10000, x=10000), 
                                       lock=False)
    
    # create and export tif with natural forests
    xr.where(da_lesiv == 11, 100, 0) \
        .rename('lesiv_nat') \
        .astype('int8') \
        .rio.to_raster(dir01 + 'lesiv_nat.tif', 
                       tiled=True, 
                       lock=Lock('rio'))


prepare_lesiv_nat()

In [ ]:
# Close dask cluster
cluster.close()

In [ ]:
# Wait for 60s for dask client to completely disconnect
time.sleep(60)

---

In [ ]:
# Import regridding function
from regrid_high_res_v1_01 import regrid_high_res, prep_tif

In [ ]:
def regrid_da(f_source, dir_target, dir_source, dir_out, 
              size_tiles, fill_value=None, olap=1):  
    """Regrid large xarray dataarrays.

    Args:
        f_source (str): The filename (without extension) of the source .tif file to be regridded.
        dir_target (str): Directory containing target grid .tif file.
        dir_source (str): Directory containing the the source  .tif file.
        dir_out (str): Directory to store the output and intermediate files.
        size_tiles (int): Size of the regridding tiles in degrees.
        fill_value (float, optional): Fill value to use in the regridding process. Defaults to None.
        olap (int, optional): Overlap size in degrees for regridding tiles. Defaults to 1.
        
    Returns:
        xarray.Dataset: The combined dataset after regridding.
    """
    # Prepare the target and source data arrays from TIFF files
    da_target = prep_tif(dir_target + 'target_grid.tif', 'target_grid')
    da_source = prep_tif(dir_source + f_source + '.tif', f_source)
    # Regridd source array to target grid
    regrid_high_res(da_target, da_source, dir_out,
                    account='bm0891', partition='compute',
                    size_tiles=size_tiles, olap=olap, fill_value = fill_value,
                    type_export='zarr', del_interm=False)

In [ ]:
# regridding
for i in ['lesiv_nat']:
    %time regrid_da(i, dir01, dir01, dir01, 10, -1, 0.1)

---

In [ ]:
xr.open_zarr(dir01 + 'ds_regridded_lesiv_nat.zarr').regridded_lesiv_nat.plot.imshow()

In [ ]:
# read regridded data
ds_lesiv = xr.open_zarr(dir01 + 'ds_regridded_lesiv_nat.zarr')

# set grid cells with natural forest >= 90% to True, all other to 0
# for all other forest areas a non-natural state is assumed
# export as zarr
xr.where(ds_lesiv.regridded_lesiv_nat >= 90, 1, 0) \
    .to_dataset(name='lesiv_nat') \
    .to_zarr(dir01 + 'ds_prep_lesiv_nat.zarr', mode='w');

---

### Check

In [ ]:
# Plot to check
for i in ['lesiv_nat']:
    fig, ax = plt.subplots(figsize=(10, 5), ncols=1, nrows=1)
    xr.open_zarr(dir01 + 'ds_prep_' + i + '.zarr')[i] \
        .plot.imshow(ax=ax)
    ax.set_title(i)